In [1]:
import stk, stko

import py3Dmol

from molecule import Molecule,show_stk_mol

from VisualizeTrajectory import TrajectoryVisualizer

In [2]:
smiles = 'O=C(O)c1ccccc1O'

In [3]:
acid = stk.BuildingBlock( smiles, [stk.CarboxylicAcidFactory()] )

show_stk_mol( acid )

stk.XyzWriter().write( acid, './acid.xyz' )

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
# Don't compute the molecule trajectory from optimization.
# This uses a precomputed trajectory file for the molecule instead.
# If you want to run the optimization again, set this to true; 
# Running the optimization will take some time and the trajectory can not be
# visualized before the operation has finished.
recomputeOptimization = False

In [5]:
if(recomputeOptimization):

    from ase.io import read, write

    atoms = read( './acid.xyz' )

    from tblite.ase import TBLite

    atoms.calc = TBLite(method="GFN1-xTB")


    from ase.optimize import BFGS

    dynamics = BFGS( atoms, trajectory = 'trajectory.traj' )

    dynamics.run( fmax=0.05 )

In [6]:
show_trajectory = TrajectoryVisualizer('./trajectory.traj')
show_trajectory.show_trajectory()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol